<a href="https://colab.research.google.com/github/mohannashahrad/Comp598_Project/blob/main/Notebooks/comp598_tfidf_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import pandas as pd
import math
import json
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from matplotlib import pyplot as plt
import collections

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Import pllotly for plots
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Define Functions

In [ ]:
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

def expandContractions(text,contractions_dict=contractions_dict):
    regex=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
    def replace(match):
        return contractions_dict[match.group(0)]
    return regex.sub(replace, text)

def plot_word_frequency_hist(data, category):
  #categories = ['infections', 'pandemic effects', 'policy', 'science', 'vaccination']
  plot_data = collections.Counter(data).most_common(20)
  keys = []
  values = []
  for i in plot_data:
    keys.append(i[0])
    values.append(i[1])
  if category == 'infections':
    title = 'Infections'
  elif category == 'policy':
    title = 'Policy' 
  elif category == 'science':
    title = 'Science'
  elif category == 'vaccination':
    title = 'Vaccination'    
  else:
    title = 'Pandemic Effects'  
  fig = px.bar(x = keys, y = values, width=600, height=600)
  fig.update_layout(yaxis_title="Frequency")
  fig.update_xaxes(title_text='Words', tickangle=-45)
  fig.update_layout(title_text=f'{title} category', title_x=0.5)
  fig.show()

# Loading the finalized dataset

In [ ]:
url = 'https://raw.githubusercontent.com/mohannashahrad/Comp598_Project/main/data/final_labeled_data.csv'
df = pd.read_csv(url)
display(df.head(30))

,category,sentiment,text
0,infections,negative,B.C. counts 10 more deaths over weekend from C...
1,infections,negative,"Always deflection, never answers. And why is t..."
2,infections,neutral,"Covid 19 Update. Confirmed 4,323, No Change. ..."
3,infections,positive,"Less cases than yesterday, but still not good...."
4,infections,neutral,"COVID-19 in Sask.: 1,047 active cases, 150 pat..."
5,infections,negative,@RebeccaDeVaney1 Will be full of “vaxxed” unte...
6,infections,neutral,Here's @stephanielevitz on how the first day b...
7,infections,negative,Your @NiagarasNewRock 3pm news update:\r\n-Ont...
8,infections,neutral,"NSW records 180 new COVID-19 cases, one death ..."
9,infections,negative,"@KellDA @davenewworld_2 Covid cases were 2,315..."


# Pre-processing

##Lowercase 

In [ ]:
df["text"] = df["text"].str.lower()
display(df.head(30))

,category,sentiment,text
0,infections,negative,b.c. counts 10 more deaths over weekend from c...
1,infections,negative,"always deflection, never answers. and why is t..."
2,infections,neutral,"covid 19 update. confirmed 4,323, no change. ..."
3,infections,positive,"less cases than yesterday, but still not good...."
4,infections,neutral,"covid-19 in sask.: 1,047 active cases, 150 pat..."
5,infections,negative,@rebeccadevaney1 will be full of “vaxxed” unte...
6,infections,neutral,here's @stephanielevitz on how the first day b...
7,infections,negative,your @niagarasnewrock 3pm news update:\r\n-ont...
8,infections,neutral,"nsw records 180 new covid-19 cases, one death ..."
9,infections,negative,"@kellda @davenewworld_2 covid cases were 2,315..."


##Expanding Contractions

In [ ]:
df['text'] = df['text'].apply(lambda x:expandContractions(x))
display(df.head(30))

,category,sentiment,text
0,infections,negative,b.c. counts 10 more deaths over weekend from c...
1,infections,negative,"always deflection, never answers. and why is t..."
2,infections,neutral,"covid 19 update. confirmed 4,323, no change. ..."
3,infections,positive,"less cases than yesterday, but still not good...."
4,infections,neutral,"covid-19 in sask.: 1,047 active cases, 150 pat..."
5,infections,negative,@rebeccadevaney1 will be full of “vaxxed” unte...
6,infections,neutral,here is @stephanielevitz on how the first day ...
7,infections,negative,your @niagarasnewrock 3pm news update:\r\n-ont...
8,infections,neutral,"nsw records 180 new covid-19 cases, one death ..."
9,infections,negative,"@kellda @davenewworld_2 covid cases were 2,315..."


## Removing the tweeter usernames

In [ ]:
df['text'] = df['text'].apply(lambda x:re.sub('@[^\s]+','', x))
display(df.head(30))

,category,sentiment,text
0,infections,negative,b.c. counts 10 more deaths over weekend from c...
1,infections,negative,"always deflection, never answers. and why is t..."
2,infections,neutral,"covid 19 update. confirmed 4,323, no change. ..."
3,infections,positive,"less cases than yesterday, but still not good...."
4,infections,neutral,"covid-19 in sask.: 1,047 active cases, 150 pat..."
5,infections,negative,will be full of “vaxxed” untested covid sprea...
6,infections,neutral,here is on how the first day back went for er...
7,infections,negative,your 3pm news update:\r\n-ontario reports 627...
8,infections,neutral,"nsw records 180 new covid-19 cases, one death ..."
9,infections,negative,"covid cases were 2,315 in canada (7 day aver..."


## Removing Hashtags

In [ ]:
df['text'] = df['text'].apply(lambda x:re.sub('#[^\s]+','', x))
display(df.head(30))

,category,sentiment,text
0,infections,negative,b.c. counts 10 more deaths over weekend from c...
1,infections,negative,"always deflection, never answers. and why is t..."
2,infections,neutral,"covid 19 update. confirmed 4,323, no change. ..."
3,infections,positive,"less cases than yesterday, but still not good...."
4,infections,neutral,"covid-19 in sask.: 1,047 active cases, 150 pat..."
5,infections,negative,will be full of “vaxxed” untested covid sprea...
6,infections,neutral,here is on how the first day back went for er...
7,infections,negative,your 3pm news update:\r\n-ontario reports 627...
8,infections,neutral,"nsw records 180 new covid-19 cases, one death ..."
9,infections,negative,"covid cases were 2,315 in canada (7 day aver..."


## Punctuation

In [ ]:
df['text'] = df['text'].apply(lambda x:re.sub(r'[^\w\s]','', x))
display(df.head(30))

,category,sentiment,text
0,infections,negative,bc counts 10 more deaths over weekend from cov...
1,infections,negative,always deflection never answers and why is the...
2,infections,neutral,covid 19 update confirmed 4323 no change dece...
3,infections,positive,less cases than yesterday but still not good s...
4,infections,neutral,covid19 in sask 1047 active cases 150 patients...
5,infections,negative,will be full of vaxxed untested covid spreade...
6,infections,neutral,here is on how the first day back went for er...
7,infections,negative,your 3pm news update\r\nontario reports 627 n...
8,infections,neutral,nsw records 180 new covid19 cases one death ht...
9,infections,negative,covid cases were 2315 in canada 7 day averag...


##Removing digits

In [ ]:
df['text'] = df['text'].apply(lambda x: re.sub(r'\d+','',x))
display(df.head(30))

,category,sentiment,text
0,infections,negative,bc counts more deaths over weekend from covid...
1,infections,negative,always deflection never answers and why is the...
2,infections,neutral,covid update confirmed no change deceased ...
3,infections,positive,less cases than yesterday but still not good s...
4,infections,neutral,covid in sask active cases patients in hospi...
5,infections,negative,will be full of vaxxed untested covid spreade...
6,infections,neutral,here is on how the first day back went for er...
7,infections,negative,your pm news update\r\nontario reports new c...
8,infections,neutral,nsw records new covid cases one death httpstc...
9,infections,negative,covid cases were in canada day average wi...


## Removing stopwords

In [ ]:
stop = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
display(df.head(30))

,category,sentiment,text
0,infections,negative,bc counts deaths weekend covid httpstcoukikhrtzr
1,infections,negative,always deflection never answers goal reduce co...
2,infections,neutral,covid update confirmed change deceased sadly r...
3,infections,positive,less cases yesterday still good solution vacci...
4,infections,neutral,covid sask active cases patients hospital cbc ...
5,infections,negative,full vaxxed untested covid spreaders guess
6,infections,neutral,first day back went erin otoole httpstcolauxzk...
7,infections,negative,pm news update ontario reports new cases covid...
8,infections,neutral,nsw records new covid cases one death httpstco...
9,infections,negative,covid cases canada day average gt eligible pop...


##Removing extra spaces

In [ ]:
df['text'] = df['text'].apply(lambda x:re.sub(' +',' ',x))
display(df.head(30))

,category,sentiment,text
0,infections,negative,bc counts deaths weekend covid httpstcoukikhrtzr
1,infections,negative,always deflection never answers goal reduce co...
2,infections,neutral,covid update confirmed change deceased sadly r...
3,infections,positive,less cases yesterday still good solution vacci...
4,infections,neutral,covid sask active cases patients hospital cbc ...
5,infections,negative,full vaxxed untested covid spreaders guess
6,infections,neutral,first day back went erin otoole httpstcolauxzk...
7,infections,negative,pm news update ontario reports new cases covid...
8,infections,neutral,nsw records new covid cases one death httpstco...
9,infections,negative,covid cases canada day average gt eligible pop...


#Calculating TF-IDF

In [ ]:
def filter_df_by_category(category):
  result = df[df['category'] == category]
  result.reset_index(drop=True, inplace=True)
  return result

def compute_tf_idf(category, data_dict, word):
    # tf = number of times the word is used in the same category
    tf = data_dict[category][word]
    # log [ (total number of categories) / (number of categoreis that use the word w)]
    idf = compute_idf(data_dict, word)
    return tf * idf

def compute_idf(data, word):
    total_num = 5
    categories_that_used_word = 0
    for c in data:
        if word in data[c]:
            categories_that_used_word += 1
    return math.log(total_num/categories_that_used_word,10)

def sort_result(result_dict, num_of_words):
    # Sorting the dictionary by value of tf-idf scores
    sorted_list = sorted(result_dict.items(), key=lambda x: x[1], reverse=True)
    result_list = []
    if len(sorted_list) < num_of_words:
        return result_list
    for i in range(num_of_words):
        result_list.append(sorted_list[i][0])
    return result_list 
def extract_phrase(input):
  result = {}
  for i in input.text:
    words = i.split()
    bi_grams = nltk.ngrams(words, 2)
    for gram in bi_grams:
      if gram in result:
        result[gram] = result[gram] + 1
      else:  
        result[gram] = 1
  return result            


In [ ]:
# Creating a dictionary of frequencies per category 

categories = ['infections', 'pandemic effects', 'policy', 'science', 'vaccination']
freq_dict = {}
for c in categories:
  freq = filter_df_by_category(c)['text'].str.split(expand=True).stack().value_counts()
  phrases = pd.Series(extract_phrase(filter_df_by_category(c)))
  #freq_df = pd.DataFrame(pd.concat([freq, phrases], axis=0)).transpose()
  freq_df = pd.DataFrame(freq).transpose()
  result = freq_df.to_dict('records')
  freq_dict[c] = result[0]
  print(result[0])
  #plot_word_frequency_hist(result[0],c) 

{'covid': 82, 'cases': 38, 'new': 25, 'vaccination': 22, 'deaths': 17, 'people': 14, 'vaccinated': 14, 'positive': 12, 'health': 10, 'fully': 10, 'bc': 9, 'via': 9, 'status': 8, 'ontario': 8, 'one': 8, 'number': 8, 'still': 8, 'death': 7, 'tests': 7, 'mp': 6, 'amp': 6, 'died': 6, 'like': 6, 'day': 6, 'reported': 6, 'year': 5, 'would': 5, 'data': 5, 'reporting': 5, 'update': 5, 'see': 5, 'pfizer': 5, 'canada': 5, 'weekend': 5, 'due': 5, 'news': 5, 'virus': 5, 'us': 5, 'erin': 4, 'today': 4, 'weeks': 4, 'reports': 4, 'case': 4, 'im': 4, 'icu': 4, 'vaxed': 4, 'official': 4, 'unknown': 4, 'otoole': 4, 'unvaccinated': 4, 'infections': 4, 'going': 4, 'without': 4, 'sorry': 4, 'safe': 4, 'kids': 4, 'rates': 4, 'first': 4, 'active': 4, 'tested': 4, 'total': 3, 'well': 3, 'always': 3, 'daily': 3, 'europe': 3, 'tory': 3, 'way': 3, 'approved': 3, 'good': 3, 'schools': 3, 'something': 3, 'parliament': 3, 'average': 3, 'policy': 3, 'rate': 3, 'spotlight': 3, 'life': 3, 'k': 3, 'loss': 3, 'officials

## Calling TF-IDF functions

In [ ]:
result_dict = {}
for c in categories:
  score_dict = {}
  for word in freq_dict[c]:
    score_dict[word] = compute_tf_idf(c, freq_dict, word)
  print(sorted(score_dict.items(), key=lambda x: x[1], reverse=True))  
  result_dict[c] = sort_result(score_dict, 10)

print(result_dict)
# Plot the scores of tf-idfs    

[('erin', 2.795880017344075), ('positive', 2.6621849953962764), ('mp', 2.387640052032226), ('tory', 2.0969100130080562), ('spotlight', 2.0969100130080562), ('loss', 2.0969100130080562), ('blah', 2.0969100130080562), ('dogs', 2.0969100130080562), ('eve', 2.0969100130080562), ('reporting', 1.989700043360188), ('weekend', 1.989700043360188), ('official', 1.5917600346881504), ('unknown', 1.5917600346881504), ('otoole', 1.5917600346881504), ('tests', 1.5529412473144946), ('condolences', 1.3979400086720375), ('goal', 1.3979400086720375), ('relatively', 1.3979400086720375), ('british', 1.3979400086720375), ('count', 1.3979400086720375), ('roman', 1.3979400086720375), ('castex', 1.3979400086720375), ('niagara', 1.3979400086720375), ('columbia', 1.3979400086720375), ('reported', 1.3310924976981382), ('total', 1.193820026016113), ('average', 1.193820026016113), ('officials', 1.193820026016113), ('fully', 0.9691001300805641), ('vaxed', 0.8873949984654255), ('without', 0.8873949984654255), ('sorry

#Plots and Results


In [ ]:
result_df = pd.DataFrame.from_dict(result_dict)
display(result_df)

,infections,pandemic effects,policy,science,vaccination
0,erin,stocks,proof,study,rd
1,positive,operation,aged,cell,spots
2,mp,dark,approves,type,detailed
3,tory,airborne,authorized,antibody,availability
4,spotlight,shipping,asks,mrna,aged
5,loss,violence,government,rna,appointments
6,blah,guilty,provincial,contraindications,book
7,dogs,piece,approve,adolescents,toronto
8,eve,women,policy,mixing,date
9,reporting,economic,authorize,immune,arm


## Sentiment Analysis


In [ ]:
import plotly.express as px
def plot_sentiment_pie(df, category):
  sentiment_count_dict = df['sentiment'].value_counts().to_dict()
  data_list = []
  neg_ratio = {}
  for i in sentiment_count_dict:
    data_list.append([i,sentiment_count_dict[i]])
  q = pd.DataFrame(data_list, columns=["Sentiment", "Count"])
  fig = px.pie(q, values='Count', names='Sentiment', title=f'{category} Category', color='Sentiment',
               color_discrete_sequence=["blue","green","red"], width=500, height=500)
  fig.show() 
def plot_neg_over_pos_sentiment_ratio(df):
  ratio_data = {}
  for c in categories:
    df_cat = df[df['category'] == c]
    sentiment_count_dict = df_cat['sentiment'].value_counts().to_dict()
    ratio_data[c] = sentiment_count_dict['negative']/sentiment_count_dict['positive']   
  keys = []
  values = []
  for i in ratio_data:
    keys.append(i)
    values.append(ratio_data[i])
  fig = px.bar(x = keys, y = values,width=500, height=500)
  fig.update_layout(
    xaxis_title="Category", yaxis_title="Ratio of negative over positive sentiments"
)
  fig.show()

In [ ]:
plot_neg_over_pos_sentiment_ratio(df)

In [ ]:
plot_sentiment_pie(df[df['category'] == 'infections'], 'Infections')
plot_sentiment_pie(df[df['category'] == 'vaccination'], 'Vaccination')
plot_sentiment_pie(df[df['category'] == 'policy'], 'Policy')
plot_sentiment_pie(df[df['category'] == 'science'], 'Science')
plot_sentiment_pie(df[df['category'] == 'pandemic effects'], 'Pandemic Effects')